In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import pandas as pd
import preprocessor as p

In [2]:
fp = "../data/REVISION DATASET_b.xlsx"
df = pd.read_excel(fp,sheet_name="data")

In [3]:
df.head()

,fullname,is_retweet,likes,replies,retweets,text,expresses a pain point,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,BRAND,Type of Pain,Subjectivity,Second category,Third category,Fourth category,Dataset
0,Emmanuel Olabode,0,1,1,1,@ifemeetstech your pr team can help bridge the...,y,42176,1434841837,612397168788406016,/olabodeEO/status/612397168788406272,1955234486,olabodeEO,gap,Operational issues,NaN,NaN,NaN,NaN,Original
1,Alviniecððâ¨,0,0,0,0,mcdonalds really bein missing uhp people food ...,y,2014-09-01 23:25:17,1409613917,506583600599662592,/ohhamazing/status/506583600599662592,2723652417,ohhamazing,mcdonalds,Product feature or quality,NaN,NaN,NaN,NaN,Original
2,Bobby,0,0,0,0,"if they thought that little of him, why was he...",y,43720,1568242832,1171921495309910016,/Bobbythegreat/status/1171921495309914112,33740752,Bobbythegreat,gap,Product feature or quality,NaN,NaN,NaN,NaN,Original
3,Elgen Bodenstien,0,0,0,0,when towns have locally owned business capita...,y,2019-09-05 23:34:22,1567726462,1169755684122086912,/bodenstien/status/1169755684122087424,1167585352464424960,bodenstien,walmart,Company's image,NaN,NaN,NaN,NaN,Original
4,Robyn,0,3,1,1,@arma_vancouver health information in records ...,y,2019-01-23 22:43:08,1548283388,1088205518886190976,/RobynCBird/status/1088205518886191104,2206030555,RobynCBird,fitbit,Company's image,NaN,NaN,NaN,NaN,Original


In [4]:
df = df[["text","expresses a pain point"]]

In [5]:
encoded_label_dict = {"n" : 0, "y" : 1}
def encode_label(x):
    return encoded_label_dict.get(x,-1)

In [6]:
df["target"] = df["expresses a pain point"].apply(lambda x: encode_label(x))

In [7]:
#p.clean('Preprocessor is #awesome 👍 https://github.com/s/preprocessor')

In [8]:
def preprocess(txt):
    return p.clean(txt)

In [9]:
df["clean_text"] = df["text"].apply(lambda x: preprocess(x))

In [10]:
df.head()

,text,expresses a pain point,target,clean_text
0,@ifemeetstech your pr team can help bridge the...,y,1,your pr team can help bridge the communication...
1,mcdonalds really bein missing uhp people food ...,y,1,mcdonalds really bein missing uhp people food ...
2,"if they thought that little of him, why was he...",y,1,"if they thought that little of him, why was he..."
3,when towns have locally owned business capita...,y,1,when towns have locally owned business capital...
4,@arma_vancouver health information in records ...,y,1,health information in records means vulnerabil...


In [11]:
df.target.value_counts()

0    3348
1    1252
Name: target, dtype: int64

In [12]:
pain_keywords_fp = "../data/pain point keywords.txt"
pain_keywords = open(pain_keywords_fp).read().strip().split("\n")
pain_keywords

['need',
 'want',
 'wish',
 'feature',
 'ask',
 'would like',
 'improve',
 'idea',
 'upgrade',
 'support',
 'problem',
 'issue',
 'help',
 'fix',
 'complain',
 'fail',
 'sucks',
 'hope',
 'not good',
 'did not',
 'missing',
 'should',
 'i hate',
 'bad',
 'bothers',
 'bothering',
 'frustrates',
 'frustrating',
 'failure']

In [13]:
def is_keyword_in_text(txt, pain_kwds):
   return any([True if kw in txt else False for kw in pain_kwds])

In [14]:
def is_spaced_keyword_in_text(txt, pain_kwds):
   return any([True if f" {kw} " in f" {txt} " else False for kw in pain_kwds])

In [15]:
#is_keyword_in_text("some water",pain_keywords)
#is_spaced_keyword_in_text("need some water",pain_keywords)

#### Train/Valid/Test Split

In [16]:
train, valid_test = train_test_split(df, test_size=0.2, shuffle=True, stratify=None, random_state=2021)
valid, test = train_test_split(valid_test, test_size=0.5, shuffle=True, stratify=None, random_state=2021)

#### Adding Inferece functions

In [17]:
def predict(data,text_col,pain_kwds):
    preds = list(data[text_col].apply(lambda x: int(is_keyword_in_text(x,pain_kwds))).values)
    return preds

In [18]:
def predict_with_space(data,text_col,pain_kwds):
    preds = list(data[text_col].apply(lambda x: int(is_spaced_keyword_in_text(x,pain_kwds))).values)
    return preds

#### Exact Match without preprocessing

In [19]:
train_preds = predict(train,"text",pain_keywords)

In [20]:
valid_preds = predict(valid,"text",pain_keywords)

In [21]:
test_preds = predict(test,"text",pain_keywords)

In [22]:
from sklearn.metrics import confusion_matrix
y_true = test.target.values
y_pred = test_preds
confusion_matrix(y_true,y_pred)

array([[  0, 332],
       [  0, 128]])

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [24]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 27.82608695652174; Precision:27.82608695652174; Recall:100.0


In [25]:
print(classification_report(y_true, y_pred, target_names=["N","Y"]))

              precision    recall  f1-score   support

           N       0.00      0.00      0.00       332
           Y       0.28      1.00      0.44       128

    accuracy                           0.28       460
   macro avg       0.14      0.50      0.22       460
weighted avg       0.08      0.28      0.12       460



/home/lazylearner/anaconda3/envs/amazonreviews/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Exact Match after preprocessing

In [26]:
train_preds = predict(train,"clean_text",pain_keywords)

In [27]:
valid_preds = predict(valid,"clean_text",pain_keywords)

In [28]:
test_preds = predict(test,"clean_text",pain_keywords)

In [29]:
from sklearn.metrics import confusion_matrix
y_true = test.target.values
y_pred = test_preds
confusion_matrix(y_true,y_pred)

array([[  4, 328],
       [  1, 127]])

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [31]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 28.47826086956522; Precision:27.912087912087912; Recall:99.21875


In [32]:
print(classification_report(y_true, y_pred, target_names=["N","Y"]))

              precision    recall  f1-score   support

           N       0.80      0.01      0.02       332
           Y       0.28      0.99      0.44       128

    accuracy                           0.28       460
   macro avg       0.54      0.50      0.23       460
weighted avg       0.66      0.28      0.14       460



#### Spaced Match without preprocessing

In [33]:
train_preds = predict_with_space(train,"text",pain_keywords)

In [34]:
valid_preds = predict_with_space(valid,"text",pain_keywords)

In [35]:
test_preds = predict_with_space(test,"text",pain_keywords)

In [36]:
from sklearn.metrics import confusion_matrix
y_true = test.target.values
y_pred = test_preds
confusion_matrix(y_true,y_pred)

array([[ 35, 297],
       [ 16, 112]])

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [38]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 31.956521739130434; Precision:27.383863080684595; Recall:87.5


In [39]:
print(classification_report(y_true, y_pred, target_names=["N","Y"]))

              precision    recall  f1-score   support

           N       0.69      0.11      0.18       332
           Y       0.27      0.88      0.42       128

    accuracy                           0.32       460
   macro avg       0.48      0.49      0.30       460
weighted avg       0.57      0.32      0.25       460



#### Spaced Match after preprocessing

In [40]:
train_preds = predict_with_space(train,"clean_text",pain_keywords)

In [41]:
test_preds = predict_with_space(valid,"clean_text",pain_keywords)

In [42]:
preds = predict_with_space(test,"clean_text",pain_keywords)

In [43]:
from sklearn.metrics import confusion_matrix
y_true = test.target.values
y_pred = test_preds
confusion_matrix(y_true,y_pred)

array([[ 38, 294],
       [ 13, 115]])

In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [45]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 33.26086956521739; Precision:28.117359413202937; Recall:89.84375


In [46]:
print(classification_report(y_true, y_pred, target_names=["N","Y"]))

              precision    recall  f1-score   support

           N       0.75      0.11      0.20       332
           Y       0.28      0.90      0.43       128

    accuracy                           0.33       460
   macro avg       0.51      0.51      0.31       460
weighted avg       0.62      0.33      0.26       460

